In [4]:
%%capture
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import huggingface_hub
huggingface_hub.login()

In [5]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
from tqdm import tqdm_notebook
import torch
from datasets import load_dataset
import numpy as np
import wandb
from evaluate import load
import os
import pandas as pd
from datasets import Dataset
import time
import evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, MT5ForConditionalGeneration
from huggingface_hub import create_repo, upload_folder

In [ ]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alinatl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Load data

In [ ]:
from datasets import load_dataset

kir = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="https://huggingface.co/datasets/openlanguagedata/oldi_seed/resolve/refs/pr/4/seed/kir_Cyrl.parquet"
)['train']

eng = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="seed/eng_Latn.parquet"
)["train"]

eng_dict = {row["id"]: row["text"] for row in eng}
merged = kir.map(lambda row: {"kir": row["text"], "eng": eng_dict.get(row["id"], None)})
train_dataset = merged.select_columns(["eng", "kir"])

print(train_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 6193
})

In [6]:
eng_ds = load_dataset("openlanguagedata/flores_plus", "eng_Latn", split="dev")
kir_ds = load_dataset("openlanguagedata/flores_plus", "kir_Cyrl", split="dev")

assert len(eng_ds) == len(kir_ds)

parallel_data = [
    {"eng": eng_ds[i]["text"], "kir": kir_ds[i]["text"]}
    for i in range(len(eng_ds))
]

test_dataset = Dataset.from_list(parallel_data)
test_dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/72.5k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/220 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/214 [00:00<?, ?it/s]

dev/eng_Latn.parquet:   0%|          | 0.00/112k [00:00<?, ?B/s]

...

dev/kir_Cyrl.parquet:   0%|          | 0.00/150k [00:00<?, ?B/s]

devtest/kir_Cyrl.parquet:   0%|          | 0.00/157k [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 997
})

## Load model

In [ ]:
checkpoint = 'google/mt5-base'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model = model.cuda()

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Train pytorch

## build dataset

In [ ]:
import torch

torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [ ]:
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'ky': '<ky>'
}

source_lang = "ky"
target_lang = "en"

max_seq_len = 128
train_batch_size = 8
eval_batch_size = 8
n_epochs = 6
logging_steps = 775
eval_steps = 775
eval_steps_blue = 775
save_model_epoch = 6
lr = 1e-4
max_norm_clip = 3
n_batches = int(np.ceil(len(train_dataset) / train_batch_size))
n_batches_eval = int(np.ceil(len(test_dataset) / eval_batch_size))
max_iters_eval= n_batches_eval
total_steps = n_epochs * n_batches
n_warmup_steps = 0
checkpoint_path = "checkpoints/" + '_epoch_'

In [ ]:
special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250102, 768)

In [ ]:
example_input_str = '<en> Менин Атым Алина!'
token_ids = tokenizer.encode(
    example_input_str, return_tensors='pt', padding='max_length',
    truncation=True, max_length=max_seq_len)
print(token_ids)

tokens = tokenizer.convert_ids_to_tokens(token_ids[0])
print(tokens)

tensor([[250100,  14152,   2249,    864,  11147,  28565,    558,    309,      1,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,    

In [ ]:
model.eval()

example_input_str = ['<en> Менин атым Алина!',
                     '<en> Менин сүйүктүү жазуучум Чыңгыз Айтматов.']
for sent in example_input_str:
    token_ids = tokenizer.encode(
        sent, return_tensors='pt', padding='max_length',
        truncation=True, max_length=max_seq_len)


    with torch.no_grad():
        generated_ids = model.generate(token_ids.to('cuda'), max_length=max_seq_len)
        decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        print(decoded_preds)
model.train()
print('----')

['<extra_id_0> а']
['<extra_id_0>, Кыргызстан']
----


In [ ]:
def encode_input_str(text, target_lang, tokenizer, seq_len,
                     lang_token_map=LANG_TOKEN_MAPPING):
  target_lang_token = lang_token_map[target_lang]

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = target_lang_token + text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]


def encode_target_str(text, tokenizer, seq_len,
                      lang_token_map=LANG_TOKEN_MAPPING):
  token_ids = tokenizer.encode(
      text = text,
      return_tensors = 'pt',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return token_ids[0]


def format_translation_data(translations, lang_token_map,
                            tokenizer, seq_len=128):
  input_text = translations[ 'kir']
  target_text = translations['eng']

  if input_text is None or target_text is None:
    return None

  input_token_ids = encode_input_str(
      input_text, target_lang, tokenizer, seq_len, lang_token_map)

  target_token_ids = encode_target_str(
      target_text, tokenizer, seq_len, lang_token_map)

  return input_token_ids, target_token_ids


def transform_batch(batch, lang_token_map, tokenizer):
  inputs = []
  targets = []

  for i in range(len(batch['kir'])):
    input_text = batch['kir'][i]
    target_text = batch['eng'][i]
    translation_set = {'kir':input_text, 'eng': target_text}

    formatted_data = format_translation_data(
        translation_set, lang_token_map, tokenizer, max_seq_len)

    input_ids, target_ids = formatted_data
    inputs.append(input_ids.unsqueeze(0))
    targets.append(target_ids.unsqueeze(0))

  batch_input_ids = torch.cat(inputs).cuda()
  batch_target_ids = torch.cat(targets).cuda()
  batch_target_ids[batch_target_ids == tokenizer.pad_token_id] = -100

  return batch_input_ids, batch_target_ids

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=train_batch_size):
  dataset = dataset.shuffle()
  for i in range(0, len(dataset), batch_size):
    raw_batch = dataset[i:i+batch_size]
    yield transform_batch(raw_batch, lang_token_map, tokenizer)

In [ ]:
train_dataset

Dataset({
    features: ['eng', 'kir'],
    num_rows: 6193
})

In [ ]:
data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, train_batch_size)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

Input shape: torch.Size([8, 128])
Output shape: torch.Size([8, 128])


## training loop

In [ ]:
from torch.optim import AdamW
from transformers import get_scheduler


optimizer = AdamW(model.parameters(), lr=lr)
scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=n_warmup_steps,
    num_training_steps=total_steps)

In [ ]:
def save_to_hf(model, tokenizer, epoch_idx, checkpoint_path):
    checkpoint_path = checkpoint_path + str(epoch_idx)
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)

    create_repo("alinatl/mT5-base-ky-en", exist_ok=True)

    upload_folder(
        folder_path=checkpoint_path,
        repo_id="alinatl/mT5-base-ky-en",
        repo_type="model",
        path_in_repo=checkpoint_path
    )

In [ ]:
def compute_grad_norm(model):
  total_norm = 0.0
  for p in model.parameters():
      if p.grad is not None:
          param_norm = p.grad.data.norm(2)
          total_norm += param_norm.item() ** 2
  grad_norm = total_norm ** 0.5
  return grad_norm

In [ ]:
def compute_metrics(model):
    start_time = time.time()
    bleu = evaluate.load("bleu")
    chrfpp = evaluate.load("chrf")

    decoded_preds_all = []
    decoded_labels_all = []

    with torch.no_grad():
        data_generator_eval = get_data_generator(test_dataset, LANG_TOKEN_MAPPING, tokenizer, eval_batch_size)
        for i, (input_batch, label_batch) in tqdm_notebook(enumerate(data_generator_eval), total=n_batches_eval):

            generated_ids = model.generate(input_batch, max_length=max_seq_len)
            label_batch = torch.where(label_batch != -100, label_batch, tokenizer.pad_token_id)

            decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(label_batch, skip_special_tokens=True)

            decoded_preds_all.extend(decoded_preds)
            decoded_labels_all.extend(decoded_labels)

            bleu.add_batch(predictions=decoded_preds_all, references=[[l] for l in decoded_labels_all])
            chrfpp.add_batch(predictions=decoded_preds_all, references=decoded_labels_all)

        bleu_result = bleu.compute()
        chrfpp_result = chrfpp.compute()

        print(f"CHRF++: {chrfpp_result['score']:.4f} | BLEU: {bleu_result['bleu']:.4f}")
        eval_runtime = time.time() - start_time
        steps_per_second = n_batches_eval / eval_runtime
        total_eval_samples = len(test_dataset)
        samples_per_second = total_eval_samples / eval_runtime

        wandb.log({
            "eval/chrf++": chrfpp_result['score'],
            "eval/bleu": bleu_result["bleu"],
        })

In [ ]:
def compute_eval_loss(model):
    start_time = time.time()
    eval_losses = []
    with torch.no_grad():

        data_generator_eval = get_data_generator(test_dataset, LANG_TOKEN_MAPPING, tokenizer, eval_batch_size)
        for input_batch, label_batch in data_generator_eval:
            outputs = model(input_ids=input_batch, labels=label_batch)
            eval_loss = outputs.loss
            eval_losses.append(eval_loss.item())
    avg_eval_loss = np.mean(eval_losses)
    print(f'Epoch: {epoch_idx+1} | Step: {batch_idx+1} | Avg. train loss: {avg_loss:.3f} | Avg. eval loss: {avg_eval_loss:.3f} | lr: {scheduler.get_last_lr()[0]}')

    eval_runtime = time.time() - start_time
    steps_per_second = n_batches_eval / eval_runtime
    total_eval_samples = len(test_dataset)
    samples_per_second = total_eval_samples / eval_runtime

    wandb.log({
        "eval/loss": avg_eval_loss,
        "eval/steps_per_second": steps_per_second,
        "eval/samples_per_second": samples_per_second,
        "eval/runtime": eval_runtime,
        "train/global_step": (epoch_idx * n_batches) + batch_idx + 1
    })


In [ ]:
wandb.init(project='NMT-ky-en',
               config={
                   'model': checkpoint,
                   "epochs": n_epochs,
                  "train_batch_size": train_batch_size,
                  "eval_batch_size": eval_batch_size,
                  "learning_rate": lr,
                  'max_seq_len': max_seq_len,
                  'logging_steps': logging_steps,
                  'eval_steps': eval_steps,
                  'eval_steps_blue': eval_steps_blue,
                  'save_model_epoch':save_model_epoch,
                  'max_norm_clip': max_norm_clip,
                  'n_batches': n_batches,
                  'n_batches_eval': n_batches_eval,
                  'max_iters_eval': max_iters_eval,
                  'total_steps': total_steps,
                  'n_warmup_steps': n_warmup_steps,
                  }
              )


losses = []
model_save_counter = 0

for epoch_idx in range(n_epochs):
    # Randomize data order
    data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, train_batch_size)

    for batch_idx, (input_batch, label_batch) in tqdm_notebook(enumerate(data_generator), total=n_batches):
        optimizer.zero_grad()

        # Forward pass
        model_out = model.forward(input_ids=input_batch, labels=label_batch)

        # Calculate loss and update weights
        loss = model_out.loss.mean()
        losses.append(loss.item())
        loss.backward()

        # Gradient clipping
        grad_norm = compute_grad_norm(model)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm_clip)

        optimizer.step()
        scheduler.step()


        # Print training update info
        if (batch_idx + 1) % logging_steps == 0:
            avg_loss = np.mean(losses[-logging_steps:])
            print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))
            wandb.log(
                {"train/loss": avg_loss,
                "train/learning_rate": scheduler.get_last_lr()[0],
                "train/global_step": (epoch_idx * n_batches) + batch_idx + 1,
                "train/epoch": epoch_idx + 1,
                "train/grad_norm": grad_norm
                })

        # eval loss
        if (batch_idx + 1) % eval_steps == 0:
            avg_loss = np.mean(losses[-eval_steps:])
            model.eval()
            compute_eval_loss(model)
            model.train()


        if (batch_idx + 1) % eval_steps_blue == 0:
            model.eval()
            compute_metrics(model)
            model.train()

/tmp/ipython-input-2868421118.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx, (input_batch, label_batch) in tqdm_notebook(enumerate(data_generator), total=n_batches):


  0%|          | 0/775 [00:00<?, ?it/s]

Epoch: 1 | Step: 775 | Avg. loss: 3.976 | lr: 8.333333333333334e-05
Epoch: 1 | Step: 775 | Avg. train loss: 3.976 | Avg. eval loss: 2.579 | lr: 8.333333333333334e-05


/tmp/ipython-input-1704726242.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input_batch, label_batch) in tqdm_notebook(enumerate(data_generator_eval), total=n_batches_eval):


  0%|          | 0/127 [00:00<?, ?it/s]

...

CHRF++: 36.8618 | BLEU: 0.0677


  0%|          | 0/775 [00:00<?, ?it/s]

Epoch: 6 | Step: 775 | Avg. loss: 1.837 | lr: 0.0
Epoch: 6 | Step: 775 | Avg. train loss: 1.837 | Avg. eval loss: 2.451 | lr: 0.0


  0%|          | 0/127 [00:00<?, ?it/s]

CHRF++: 36.8577 | BLEU: 0.0651


In [ ]:
save_to_hf(model, tokenizer, str(epoch_idx+1), checkpoint_path)
print(f'Saving final model at {epoch_idx+1} epoch with test loss of {loss:.3f}')

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t/checkpoints/_epoch_6/spiece.model: 100%|##########| 4.31MB / 4.31MB            

  ...checkpoints/_epoch_6/tokenizer.json:  99%|#########8| 16.1MB / 16.4MB            

  ...ckpoints/_epoch_6/model.safetensors:   0%|          |  553kB / 2.33GB            

Saving final model at 6 epoch with test loss of 0.520
